<h2>Demo from a pdf. Replace pdf_content[0] in the first call of the writer to make it functionnal</h2>

# Imports, init modèle de langue et du parser

In [ ]:
input = "Ceci est un ensemble de chunks concaténés"

from langchain_mistralai import ChatMistralAI
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser

llm = ChatMistralAI(api_key = "ImsUHfFLA6OjlX6mARbnM1YcDOy7ujsq")

output_parser = StrOutputParser()

AIMessage(content='Langsmith is a language translation and localization company, and it can help with testing in the following ways:\n\n1. Language Testing: Langsmith can ensure that the translation of your software, website, or application is accurate and free of errors. They can also test for linguistic consistency, cultural appropriateness, and compliance with industry-specific terminology.\n2. Functional Testing: Langsmith can test the functionality of your localized software, website, or application to ensure that it works as expected in different languages. This includes testing for character display, text wrapping, and other language-specific issues that can affect the user experience.\n3. Localization Testing: Langsmith can perform localization testing to ensure that your software, website, or application is adapted to the cultural, legal, and technical requirements of the target market. This includes testing for date and time formats, currency symbols, and other regional diffe

<h2>Definition of the summarizer and its memory</h2>

In [80]:
writerPrompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are the redactor. Given a chapter of a book, you are able to write a small course in less than 300 words, that is very ludic and interesting about the topic of the corresponding document.
    You need to adapt the informations that you teach based on 3 levels of difficulty:
    - Newbie
    - Intermediate
    - Advanced
    """
    ),
    ("user", "{input}"),
    ])

# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="writer_history")

writerConversation = LLMChain(
    llm=llm,
    prompt=writerPrompt,
    verbose=True,
    memory=memory
)

firstSummary = writerConversation.run('You are going to summarize for a Newbie. Here is the information : ' + input)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the redactor. Given a chapter of a book, you are able to write a small course in less than 300 words, that is very ludic and interesting about the topic of the corresponding document.
    You need to adapt the informations that you teach based on 3 levels of difficulty:
    - Newbie
    - Intermediate
    - Advanced
    
Human: You are going to summarize for a Newbie. Here is the information : CLASSE DE TERMINALE..................................................................... 5
COURS D'HISTOIRE............................................................................. 5
S............................................................................................................. 5
Introduc tion : le Monde  en 1945 ( 2 ou 3 he ures)..................................................................... 6
I. Un monde  boul eversé...........................................................................

<h2>Definition of the difficulty critic and answer of the writer</h2>

In [90]:
difficultyCriticPrompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are the critic. You are part of a course redaction team experts that creates very ludic and interesting courses based on real books to make them more readable and interactive. Your role in this group is to evaluate the course written by the redactor so that it respects the level of difficulty asked by the user.
    Ther 3 levels of difficulty:
    - Newbie
    - Intermediate
    - Advanced

    You will critic the course given by the redactor evaluate if it respects the difficulty level and explain why.
     """
     ),
    ("user", "{input}"),
    ])


difficultyCriticChain = difficultyCriticPrompt | llm | output_parser
difficultyCriticOutput = difficultyCriticChain.invoke(firstSummary)

In [91]:
summaryDifficultyCritiqued = writerConversation.run(difficultyCriticOutput)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the redactor. Given a chapter of a book, you are able to write a small course in less than 300 words, that is very ludic and interesting about the topic of the corresponding document.
    You need to adapt the informations that you teach based on 3 levels of difficulty:
    - Newbie
    - Intermediate
    - Advanced
    
Human: Overall, the course seems to be appropriate for the "Newbie" level of difficulty. Here are some positive aspects and areas for improvement:

Positive aspects:

* The course starts with a clear and concise definition of the Cold War, which is appropriate for someone who is new to the topic.
* The course explains the differences between the Communist government of the Soviet Union and the democratic government of the United States, which is important for understanding the political tension between the two superpowers.
* The course introduces the concept of "containment" and explains h

<h2>Definition of the style critic and answer of the writer</h2>

In [93]:
styleCriticPrompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are the critic. You are part of a course redaction team experts that creates very ludic and interesting courses based on real books to make them more readable and interactive. Your role in this group is to evaluate the course written by the redactor so that it is ludic and interesting.
    You will critic the course given by the redactor, evaluate if it is ludic and interesting and explain why.
     """
     ),
    ("user", "{input}"),
    ])


styleCriticChain = styleCriticPrompt | llm | output_parser
styleCriticOutput = styleCriticChain.invoke(summaryDifficultyCritiqued)

In [94]:

summaryStyleCritiqued = writerConversation.run(styleCriticOutput)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the redactor. Given a chapter of a book, you are able to write a small course in less than 300 words, that is very ludic and interesting about the topic of the corresponding document.
    You need to adapt the informations that you teach based on 3 levels of difficulty:
    - Newbie
    - Intermediate
    - Advanced
    
Human: Overall, this course seems well-structured and suitable for a beginner audience. The use of simple language and clear explanations makes the complex topic of the Cold War more accessible to newbies. Here are some specific strengths and areas for improvement:

Strengths:

* The course starts with a clear definition of the Cold War and its context, which is helpful for setting the stage.
* The introduction of key concepts, such as "containment," is done in a way that is easy to understand and remember.
* The mention of proxy wars provides a more nuanced understanding of the Cold War, 

In [95]:
print(summaryStyleCritiqued)

'Course Title: The Cold War: A Fun and Friendly Introduction (300 words)\n\nNewbie:\nWelcome to our course on the Cold War! The Cold War was a state of political and military tension after World War II, between the Western Bloc (led by the United States) and the Eastern Bloc (led by the Soviet Union). It lasted from 1947 to 1991 and never resulted in direct military action between the two superpowers, but it did lead to numerous proxy wars around the world. The main goal of the Western Bloc was to "contain" the Communist ideology of the Soviet Union from spreading to other parts of the world.\n\nIntermediate:\nNow that we have a basic understanding of the Cold War, let\'s dive a little deeper. Containment was a United States policy using numerous strategies to prevent the spread of communism abroad. The policy was a response to the Soviet Union\'s efforts to spread communism and was the centerpiece of U.S. foreign policy from the late 1940s through the 1980s. The Cold War was also char